In [1]:
# === Robust baseline + triadic scan (Fraction-first, no quantize) ===
from fractions import Fraction
from decimal import Decimal, getcontext

# ----- helpers -------------------------------------------------------
def fib_word(n:int) -> str:
    s = "0"
    while len(s) < n:
        s = "".join("01" if c=="0" else "0" for c in s)  # morphism 0->01, 1->0
    return s[:n]

def complement(bits:str) -> str:
    return "".join("1" if b=="0" else "0" for b in bits)

def thue_morse(n:int) -> str:
    # TM[i] = parity(popcount(i)), starting at i=0
    return "".join("1" if bin(i).count("1") & 1 else "0" for i in range(n))

def digits_to_fraction(bits:str) -> Fraction:
    # Interpret bits as base-10 digits after the decimal point: 0.b1 b2 b3...
    # Build exact integer numerator then scale by 10^N
    num = 0
    for ch in bits:
        num = num*10 + (ch == "1")
    den = 10**len(bits)
    return Fraction(num, den)

def as_decimal(frac: Fraction, places: int = 40) -> str:
    # Fixed-point string without scientific notation; no quantize
    getcontext().prec = places + 10
    q = Decimal(frac.numerator) / Decimal(frac.denominator)
    return format(q, f".{places}f")

def abs_err(frac: Fraction, target_dec: Decimal) -> Decimal:
    getcontext().prec = 50
    x = Decimal(frac.numerator) / Decimal(frac.denominator)
    return abs(x - target_dec)

# ----- config --------------------------------------------------------
N = 2000                          # digits from each sequence
alpha_ref = Decimal("0.0072973525693")  # reference α
bases_to_try = [7, 8, 9, 10, 12]  # kept for parity with older reports

# ----- sequences -> exact decimal-digit Fractions --------------------
F_bits  = fib_word(N)
R_bits  = complement(F_bits)
TM_bits = thue_morse(N)

DF  = digits_to_fraction(F_bits)
DR  = digits_to_fraction(R_bits)
DTM = digits_to_fraction(TM_bits)

# ----- 1/9 identity (finite prefix) ----------------------------------
finite_ninth = Fraction(10**N - 1, 9 * 10**N)   # 0.111... (N digits)
assert DF + DR == finite_ninth, "F+R must equal 0.111...(N)"
limit_gap = (DF + DR) - Fraction(1,9)           # = -1/(9*10^N) exactly

print("=== BASELINE (exact) ===")
print(f"digits_used N                : {N}")
print(f"DF + DR (finite)             : {as_decimal(DF+DR, 40)}")
print(f"1/9 (limit)                  : {Decimal(1)/Decimal(9)}")
print(f"finite - 1/9 (exact)         : {limit_gap}  (~{as_decimal(limit_gap, 6)})")

# ----- α* and triadic scan -------------------------------------------
def alpha_star_from(frac: Fraction, k: int) -> Fraction:
    # α*(k) = (3^k / 1000) * D(sequence)
    return Fraction(3**k, 1000) * frac

def triadic_scan(frac: Fraction, kmin=0, kmax=12):
    rows = []
    for k in range(kmin, kmax+1):
        est = alpha_star_from(frac, k)
        err = abs_err(est, alpha_ref)
        rows.append((k, 3**k, est, err))
    rows.sort(key=lambda t: t[3])  # by error
    return rows

F_scan  = triadic_scan(DF)
TM_scan = triadic_scan(DTM)
R_scan  = triadic_scan(DR)

def summarize(label, scan_rows, top=3):
    print(f"\n-- {label}: best |α* - α| (k from 0..12) --")
    for k, m, est, err in scan_rows[:top]:
        print(f"k={k:<2} m={m:<4}  est≈{as_decimal(est, 15)}  |Δ|≈{err}")

summarize("F",  F_scan)
summarize("R",  R_scan)
summarize("TM", TM_scan)

# ----- α* at k=6 (729) and step size --------------------------------
k_star = 6
alpha_star_F = alpha_star_from(DF, k_star)      # exact Fraction
resid_F = abs_err(alpha_star_F, alpha_ref)
step = Fraction(1,1000) * DF                    # one-click step = DF/1000 (exact)

print("\n=== α* at k=6 (m=729) ===")
print(f"α* (F)                : {as_decimal(alpha_star_F, 36)}")
print(f"|α* - α|              : {resid_F}")
print(f"sub-step = DF/1000    : {as_decimal(step, 15)}")
print(f"ratio resid/step      : {(Decimal(resid_F) / (Decimal(step.numerator)/Decimal(step.denominator))):.6e}")


=== BASELINE (exact) ===
digits_used N                : 2000
DF + DR (finite)             : 0.1111111111111111111111111111111111111111
1/9 (limit)                  : 0.11111111111111111111111111111111111111111111111111
finite - 1/9 (exact)         : -1/900000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000